# 上下文管理器
- manage resources

**优秀博客推荐**
- [Python with Context Managers](https://jeffknupp.com/blog/2016/03/07/python-with-context-managers/)

以打开文件为例，理解上下文管理器的必要性。系统中的文件描述符是有限的，在实际的编程中，被打开的文件描述符需要及时到进行close来释放相关的资源，否则会导致资源耗尽从而出现问题。

In [9]:
files = []
for x in range(100000):
    files.append(open('foo.txt', 'w'))

OSError: [Errno 24] Too many open files: 'foo.txt'

## 创建上下文管理器

最简单的方式是在class中添加两个方法：
- `__enter__()` 返回被管理的资源，例如文件描述符
- `__exit__()`进行资源清理工作，无返回值。

In [12]:
'''
创建上下文管理器
'''
class File():
    def __init__(self, fileName, mode):
        self._fileName = fileName
        self._mode = mode
    def __enter__(self):
        self.file = open(self._fileName, self._mode)
        return self.file
    def __exit__(self, *args):
        self.file.close()

'''
测试
    同样地打开多个文件，并没有显式地调用close()
    可是，通过上线文管理器，并没有出现问题。
'''
files = []
for i in range(10000):
    with File('foo.txt', 'w') as f:
        f.write("foo")
        files.append(f)

通过contextlib来创建上下文管理器

其中，yield相当于__enter__(),yield之后的语句相当于__exit__()语句。

In [20]:
from contextlib import contextmanager

'''
创建上下文管理器
'''
@contextmanager
def openFile(fileName, mode= "r"):
    file = open(fileName, mode)
    yield file
    file.close()

'''
测试
    打开10000个文件，非显式代用close(),查看是否会成功。
'''
files = []
for i in range(10000):
    with openFile('foo.txt', 'w') as f:
        f.write("foo")
        files.append(f)

for file in files:
    if not file.closed:
        print("File's not closed")

通过继承contextlib.ContextDecorator创建上下文管理器

In [24]:
from contextlib import ContextDecorator

class makeParagraph(ContextDecorator):
    def __enter__(self):
        print("<p>")
        return self
    
    def __exit__(self, *argc):
        print("<p>")

@makeParagraph()
def createHtml(word):
    print(word)

createHtml("This is a paragraph.")

<p>
This is a paragraph.
<p>


## 可用的上下文管理器

- threading的Lock objects
- zipfile.ZipFiles
- subprocess.Popen
- tarfile.TarFile
- telnetlib.Telnet
- pathlib.Path

In [ ]:
from threading import Lock
lock = Lock()

def do_something_dangerous():
    with lock:
        raise Exception('oops I forgot this code could raise exceptions')

try:
    do_something_dangerous()
except:
    print('Got an exception')
#lock.acquire()
print('Got here')